In [ ]:
#CASO 2: Predicción de la demanda de 5 productos top de una empresa
import pyodbc
import pandas as pd
import numpy as np
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta
import matplotlib.pylab as plt
%matplotlib inline
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import IsolationForest
import sqlalchemy

plt.rcParams['figure.figsize' ] = (16, 9)
plt.style.use('fast')


In [ ]:
sql_server="""
DRIVER={ODBC Driver 17 for SQL Server};
Server=US3RN4M3;
database=demo_prediccion;
Trusted_connection=yes;
"""

In [ ]:
query_top5_productos = """
SELECT TOP 5
[H].[id_DimProducto] AS id,
[P].[producto] AS producto
FROM [demo_prediccion].[dbo].[hechos] AS [H]
INNER JOIN [demo_prediccion].[dbo].[Dim_productos] AS [P] ON [P].[id] = [H].[id_DimProducto]
GROUP BY [H].[id_DimProducto], [P].[producto]
ORDER BY SUM([H].[cantidad]) DESC, [H].[id_DimProducto] ASC

"""

#fallando... componer

# query = """
# DECLARE @cols AS NVARCHAR(MAX)
# DECLARE @query AS NVARCHAR(MAX)

# SET @cols = STUFF((SELECT 'SUM(CASE WHEN [S].[id] = '+ CONVERT(NVARCHAR(10), [SUB]. [id]) + ' THEN [H].[cantidad] ELSE 0 END) AS [' + [SUB].[sucursal] + ']'
# FROM(
# SELECT
# [S].[id] AS id,
# [S].[sucursal] AS sucursal
# FROM [demo_prediccion].[dbo].[Dim_sucursales] AS [S]
# ORDER BY [S].[id] offset 0 rows
# ) AS SUB
# FOR XML PATH(''), TYPE).value('.', 'NVARCHAR(MAX)'), 1,2,'')

# SET @query = '
# SELECT
# [F].[fecha] AS fecha, ' + @cols + '
# FROM
# [demo_prediccion]. [dbo]. [hechos] AS [H]
# INNER JOIN [demo_prediccion].[dbo].[Dim_fechas] AS [F] ON [F].[id] = [H].[id_DimFechas]
# INNER JOIN [demo_prediccion].[dbo].[Dim_productos] AS [P] ON [P].[id] = [H].[id_DimProducto]
# INNER JOIN [demo_prediccion].[dbo].[Dim_sucursales] AS [S] ON [S].[id] = [H].[id_DimSucursal]
# WHERE
# [P].id = ''"""+'{p0}'+"""''
# GROUP BY 
# [F].[fecha]
# ORDER BY
# [F].[fecha] ASC'
# """
# query = """
# SELECT
# [F]. [Fecha] AS fecha,
# SUM([H]. [cantidad]) AS cantidad
# FROM
# [demo_prediccion]. [dbo]. [hechos] AS [H]
# INNER JOIN [demo_prediccion]. [dbo]. [Dim_fechas] AS [f] ON [H].[id_DimFechas] = [F].[id]
# GROUP BY [F].[Fecha]
# """

#Preelimminar
query = """
DECLARE @cols AS NVARCHAR(MAX);
DECLARE @query AS NVARCHAR(MAX);

-- Construcción dinámica de las columnas
SET @cols = STUFF(
    (SELECT ', SUM(CASE WHEN [S].[id] = ' + CONVERT(NVARCHAR(MAX), [SUB].[id]) + 
            ' THEN [H].[cantidad] ELSE 0 END) AS [' + [SUB].[sucursal] + ']'
     FROM (
        SELECT [S].[id] AS id, [S].[sucursal] AS sucursal
        FROM [demo_prediccion].[dbo].[Dim_sucursales] AS [S]
        ORDER BY [S].[id] OFFSET 0 ROWS
     ) AS SUB
     FOR XML PATH(''), TYPE).value('.', 'NVARCHAR(MAX)'), 
     1, 2, ''
);

-- Construcción de la consulta dinámica
SET @query = '
SELECT
    [F].[fecha] AS fecha, ' + @cols + '
FROM
    [demo_prediccion].[dbo].[hechos] AS [H]
    INNER JOIN [demo_prediccion].[dbo].[Dim_fechas] AS [F] ON [F].[id] = [H].[id_DimFechas]
    INNER JOIN [demo_prediccion].[dbo].[Dim_productos] AS [P] ON [P].[id] = [H].[id_DimProducto]
    INNER JOIN [demo_prediccion].[dbo].[Dim_sucursales] AS [S] ON [S].[id] = [H].[id_DimSucursal]
WHERE
    [P].[id] = ''"""+'{p0}'+"""''
GROUP BY 
    [F].[fecha]
ORDER BY
    [F].[fecha];';

-- Ejecutar la consulta dinámica
EXEC sp_executesql @query;

"""

In [ ]:
#Configuración para el entrenamiento (hace referencia a dias)
PASOS = 31

In [ ]:
#Opcion 1: Permitir que pueda ser modificada la sentencia, y mantenrr el try para evitar de que
# el programa se rompa

def get_sqlconnection(config_sqlServer):
    status = "inicializando...."
    try: 
        connection = pyodbc.connect(sql_server)
        status = "Conexion establecida satisfactoriamente"
    except Exception as e:
        status = "Error al establecer la conexión:"+e
    print(status)
    return connection

#Original:
# def get_sqlconnection(config_sqlServer):
#     connection = pyodbc.connect(config_sqlServer)
#     print("Se ha conectado a la base de datos")
#     return connection


In [ ]:
def set_index_datetime(data):
    if str(type(data) == "<class 'pandas.core.frame.DataFrame'>"):
        # data.sort_values('fecha', inplace=True)
        for column in data.columns: 
            try: 
                pd.to_datetime(data[column])
                data.set_index(column,inplace=True)
                return data
            except Exception as e:  
                pass
    else: 
        return 0
    
# def set_index_datetime(data):
#     if isinstance(data, pd.DataFrame):
#         for column in data.columns:
#             try:
#                 if column == 'fecha':
#                     data[column] = pd.to_datetime(data[column])
#                     data.set_index(column, inplace=True)

#                 data = data.groupby('fecha').sum().reset_index()
#                 data.dropna(inplace=True)
#                 return data
#                 # data[column] = pd.to_datetime(data[column])
#                 # data.set_index(column, inplace=True)
#                 # return data
#             except Exception as e:
#                 print(f"Error al convertir la columna {column} a datetime: {e}")
#                 pass
#     else:
#         return 0

In [ ]:
def series_to_supervised(data, n_in=1, n_out = 1, dropnan = True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    for i in range(n_in,0,-1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1,i)) for j in range(n_vars)]
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var&d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
def create_x_y_train(data):
    values = data.values
    values = values.astype('float32')
    scaler = MinMaxScaler(feature_range=(-1, 1))
    values= values.reshape(-1, 1)
    scaled = scaler.fit_transform(values)
    reframed = series_to_supervised(scaled, PASOS, 1)
    values = reframed.values
    n_train_days = int(len(data)) - (30+PASOS)
    train = values[:n_train_days, :]
    test = values[n_train_days:, :]
    x_train, y_train = train[:, :- 1], train[:, -1]
    x_val, y_val = test[:, :- 1], test[:, -1]
    x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1]))
    x_val = x_val.reshape((x_val.shape[0], 1, x_val.shape[1]))
    return x_train, y_train, x_val, y_val, scaler, values

In [ ]:
def crear_modeloFF():
    model = Sequential()
    model.add(Dense(PASOS, input_shape=(1,PASOS),activation='tanh'))
    model.add(Flatten())
    model.add(Dense(1, activation='tanh'))
    model.compile(loss='mean_absolute_error', optimizer='Adam',metrics=['mse' ])
    model.summary()
    return model

In [ ]:
def entrenar_modelo(x_train, y_train, x_val, y_val, scaler, values, data, model) :
    EPOCHS = 100
    model.fit(x_train, y_train, epochs=EPOCHS, validation_data=(x_val, y_val), batch_size=PASOS)
    model.predict(x_val)
    ultimosDias = data[data.index[int(len(data)*0.70)]:]
    values = ultimosDias.values
    values = values.astype('float32' )
    values = values.reshape(-1, 1)
    scaled = values
    reframed = series_to_supervised(scaled, PASOS, 1)
    reframed.drop(reframed.columns[[12]], axis=1, inplace=True)
    values = ultimosDias.values
    values = values.astype('float32' )
    values = values.reshape(-1, 1)
    scaled = scaler.fit_transform(values)
    reframed = series_to_supervised(scaled, PASOS, 1)
    reframed.drop(reframed.columns[[12]], axis=1, inplace=True)
    values = reframed.values
    x_test = values[len(values)-1:, :]
    x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1]) )
    return model, x_test

In [ ]:
def agregarNuevoValor(x_test, nuevoValor):
    for i in range(x_test.shape[2]-1):
        x_test[0][0][i] = x_test[0][0][i+1]
    x_test[0][0][x_test. shape[2]-1] = nuevoValor
    return x_test

In [ ]:
def eliminar_anomalias(dtaframe):
    dataFrame_anomalias = dtaframe.copy()
    modeloIsolation = IsolationForest(contamination=0.05)
    modeloIsolation.fit(dataFrame_anomalias)
    anomalias = modeloIsolation.predict(dataFrame_anomalias)
    dtaframe['anomalias' ] = anomalias
    dataFrameSinAnomalias = dtaframe[dtaframe['anomalias' ] != -1]
    dataFrameSinAnomalias = dataFrameSinAnomalias.drop('anomalias', axis=1)
    return dataFrameSinAnomalias

In [ ]:
#El Core del sistema
with get_sqlconnection(sql_server) as cursor:
    productos = pd.read_sql_query(query_top5_productos, cursor)
    # print(productos)
    # productos = set_index_datetime(productos)
    table_future_data = []
    model = crear_modeloFF()
    print(productos)
    for id in productos.id:
        # print("iniciando....")
        cadena = query.format(p0=str(id))
        # print(str(id))
        # print(cadena)
        datos = pd.read_sql_query(cadena, cursor)
        datos = set_index_datetime(datos)
        last_day = datos.index.max() + timedelta(days=1)
        future_days = [last_day + timedelta(days=i) for i in range(PASOS)]
        future_data = pd.DataFrame(future_days)
        future_data.columns = ['fecha']

In [ ]:

        for column in datos.columns:
            data = datos.filter([column])
            data.set_index(datos.index, inplace=True)
            data = eliminar_anomalias(data)
            x_train, y_train, x_val, y_val, scaler, values = create_x_y_train(data)
            model, x_test = entrenar_modelo(x_train, y_train, x_val, y_val, scaler, values, data, model)
            results = []
            for i in range(PASOS):
                parcial = model.predict(x_test)
                results.append(parcial[0])
                x_test = agregarNuevoValor(x_test, parcial[0])
            adimen = [x for x in results]
            inverted = scaler.inverse_transform(adimen)
            y_pred = pd.DataFrame(inverted.astype(int))
            future_data[column]= inverted.astype(int)
            del data, x_train, y_train, x_val, y_val, scaler, values, x_test, results, adimen, inverted, y_pred
            # future_data = set_index_datetime(future_data)
        table_future_data.append(future_data)
        



In [ ]:
# # table_future_data
pd.options.display.max_columns = None
table_future_data